In [ ]:
{'Client ID': 'Syv7stn-Rd-mtrudAbRvuaqMUb_AgdSirVvOb9XofEnxQ4_oYlVzptzilTlFhbcq',
    'Client Secret':'A0fegVhhiemDPVDdDdtYkW-8j9GF-d6pFv4_-_sbuX7Hm3yOzqlPL-HRof0bzHYu9x-CaYc0ak9AMQ3ZIYt_Ig',
'Client Access Token': 'HIs4ia8HMYMSYsZVHrYSFdAhcum7gHC0ZWMPa5pXJOoQUhd86oe6maD-BkIcKrnz'


}

In [ ]:
export GENIUS_CLIENT_ACCESS_TOKEN="HIs4ia8HMYMSYsZVHrYSFdAhcum7gHC0ZWMPa5pXJOoQUhd86oe6maD-BkIcKrnz"

### Importing necessary modules and files

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import string
from nltk import sent_tokenize, word_tokenize

/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python

In [11]:
# loading data

df = pd.read_csv('lyrics.csv')
billboard = pd.read_csv('billboards.csv')

In [12]:
#df = pd.read_csv('lyrics.csv')
# after looking at the first element of the lyrics column, I need to be remove all the spacing and annotations in 
# the csv database as well as the scraped lyrics, because its not consistent with the other elements in the lyrics album
df['song'] = df.song.str.replace('-', " ")
df['artist'] = df.artist.str.replace('-', " ")
df['lyrics'] = df.lyrics.str.replace('\n', ' ')
df.head()

,index,song,year,artist,genre,lyrics
0,0,ego remix,2009,beyonce knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut..."
1,1,then tell me,2009,beyonce knowles,Pop,"playin' everything so easy, it's like you seem..."
2,2,honesty,2009,beyonce knowles,Pop,If you search For tenderness It isn't hard to ...
3,3,you are my rock,2009,beyonce knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a..."
4,4,black culture,2009,beyonce knowles,Pop,"Party the people, the people the party it's po..."


In [13]:
# cool after I finish cleaning this dataset, i need to figure out how to find the lyrics for the songs from my
# previous project. let me load the dataset into this project repo first

billboard = pd.read_csv('billboards.csv')
billboard.Song = billboard.Song.str.replace('>', 'than')
billboard.Artist = billboard.Artist.str.replace(' x ', ', ')
billboard.drop('num_weeks', inplace = True, axis = 1)
# billboard.head(20)

billboard_dict = billboard.to_dict('records')
billboard_dict[:5]


[{'Artist': '2 Chainz', 'Song': '4 AM'},
 {'Artist': '2 Chainz', 'Song': "It's A Vibe"},
 {'Artist': '2 Chainz', 'Song': 'Proud'},
 {'Artist': '2 Chainz, Gucci Mane, Quavo', 'Song': 'Good Drank'},
 {'Artist': '2 Chainz, Drake', 'Song': 'Bigger than You'}]

### Testing out the thought process to get to the final product

Notes:

1. punctuation like $ will be replaced by '-' in the url

1. multiple artists will be in the url, so maybe i need to get all of the artists from the spotify api? put in a try/except so that i can keep track of all the songs that didn't work and do that later for a smaller subset of songs



In [ ]:
from nltk import sent_tokenize, word_tokenize


# grabbing the data from the url
artist = 'Tessa Violet'
artist_url = artist.lower().replace(' ', '-')

source_code = requests.get(f'https://genius.com/{artist_url}-crush-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()

#cleaning

lyrics = lyrics.replace('\n', '. ')
sentences = sent_tokenize(lyrics)
cleaned = [sentence for sentence in sentences if ('bridge' not in sentence.lower()) and ('verse' not in sentence.lower()) and ('chorus' not in sentence.lower()) and len(sentence) >1]

data = [artist, song, cleaned]
key =['artist', 'song', 'lyrics']
song_dict = dict(zip(key, data))
print(song_dict)

In [ ]:

artist = 'Kanye West'
artist_url = artist.replace(' ', '-')

song = 'All of the Lights'
song_url = song.replace(' ', '-')


source_code = requests.get(f'https://genius.com/{artist_url}-{song_url}-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()

#cleaning

lyrics = lyrics.replace('\n', '. ')
sentences = sent_tokenize(lyrics)
cleaned = [sentence for sentence in sentences if 
           ('produced' not in sentence.lower()) and
           ('intro' not in sentence.lower()) and
           ('outro' not in sentence.lower()) and
           ('bridge' not in sentence.lower()) and 
           ('verse' not in sentence.lower()) and 
           ('chorus' not in sentence.lower()) and len(sentence) >1]
cleaned

#it would be nice to all of this in a dictionary that i can just put into an array and then turn into a pd dataframe

data = [artist, song, cleaned]
key =['artist', 'song', 'lyrics']
song_dict = dict(zip(key, data))
# print(song_dict) #yay it worked
# please note it takes like a second for one song...

In [ ]:
# https://genius.com/2-chainz-bigger-than-you-lyrics
    
source_code = requests.get('https://genius.com/2-chainz-bigger-than-you-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()
print((lyrics)) 

In [ ]:
billboard_dict

In [17]:
# i need to building a function that takes the billboard songs and just scrapes for songs!

def scrape_genius(artist, song):
    
    #artist = 'Kanye West'
    artist = artist.replace("'", '')
    song = song.replace("'", '')
    artist_url = artist.replace(' ', '-')
    #artist_url = artist.replace(',-', '-')
    #artist_url = artist.replace(',', '')

    #song = 'All of the Lights'
    song_url = song.replace(' ', '-')

    url = f'https://genius.com/{artist_url}-{song_url}-lyrics'
    # print(url)
    source_code = requests.get(url)
    soup = BeautifulSoup(source_code.text, 'html5lib')
    lyrics = soup.find('div', class_='lyrics').text.strip()

    #cleaning

    lyrics = lyrics.replace('\n', '. ')
    sentences = sent_tokenize(lyrics)
    cleaned = [sentence for sentence in sentences if 
               ('produced' not in sentence.lower()) and
               ('intro' not in sentence.lower()) and
               ('outro' not in sentence.lower()) and
               ('bridge' not in sentence.lower()) and 
               ('verse' not in sentence.lower()) and 
               ('chorus' not in sentence.lower()) and len(sentence) >1]
    cleaned

    #it would be nice to all of this in a dictionary that i can just put into an array and then turn into a pd dataframe

    data = [artist, song, cleaned]
    key =['artist', 'song', 'lyrics']
    song_dict = dict(zip(key, data))
    return song_dict

In [18]:
len(billboard_dict)

828

### Finally Scraping

In [19]:
smaller = billboard_dict[:20]

lyrics = []
missing = []
for record in smaller:
    song = record['Song']
    artist = record['Artist']
    artist = artist.replace(', ', ' ')
    artist = artist.replace('.', '')
    song = song.replace('.', '')
    #print(artist, song)
    # artist = record['Artist']
    try:
        lyrics.append(scrape_genius(artist, song))
    except:
        print(artist, song)
        missing.append(record)
        pass
    #print(lyrics)
    #print('done')

In [20]:
len(missing)

20

In [16]:
len(lyrics)

0

In [ ]:
billboard_lyrics = pd.DataFrame(lyrics)

In [3]:
path_names = glob('second_round/*.pkl')
len(path_names)

14406

In [9]:
song_df = pd.DataFrame()
missing = []

for path in path_names:
    try:
        
        song = pd.DataFrame(pd.read_pickle(str(path)))
        combined = song.groupby(['artist', 'song'])['lyrics'].apply(' '.join).reset_index()
        song_df = song_df.append(combined)
    except:
        missing.append(path)
        print(missing)
        #break
    
song_df

,artist,song,lyrics
0,The Cascades,The Last Leaf,The last leaf clings to the bough. Just one le...
0,Keith Urban,Youll Think Of Me,I woke up early this morning around 4am. With ...
0,BJ Thomas,Mama,Who's the one who tied you shoe when you were ...
0,The Mixtures,Pushbike Song,Riding along my pushbike honey. When I noticed...
0,Greg Kihn Band,Jeopardy,Oh Oh Oh Oh. Where were you when I needed you?...
0,Expose,I Wish The Phone Would Ring,More than ever you're on my mind. Being lonely...
0,Carpenters,I Need To Be In Love,The hardest thing I've ever done. Is keep beli...
0,Soul II Soul,A Dreams A Dream,"I can see, i can see, i can see, i can see. I ..."
0,Vitamin Z,Burning Flame,Walking out tonight in the street lights I can...
0,Herbie Hancock,Chameleon,[Instrumental]


In [22]:
missing = [path for path in path_names if 'Copy1' in path]

#print(missing[0])


missing_df = pd.DataFrame(pd.read_pickle(str(missing[0])))
song_df= song_df.append(missing_df)
pd.to_pickle(song_df, 'scraped_lyrics_816.pkl')

In [48]:
song_df = pd.DataFrame(pd.read_pickle('scraped_lyrics_816.pkl'))
all_billboard_songs = (pd.DataFrame(pd.read_pickle('unique_songs.pkl')).drop('num_weeks', axis = 1)).rename(columns = {
    'Artist':'artist',
    'Song':'song'})

all_billboard_songs.artist = all_billboard_songs.artist.str.replace("'", '')
all_billboard_songs.song = all_billboard_songs.song.str.replace("'", '')
    
combined = all_billboard_songs.merge(song_df, how = 'left').drop_duplicates(['artist', 'song'])

combined.head(10)

,artist,song,lyrics
0,Rick Springfield,Jessies Girl,Jessie is a friend. Yeah I know he's been a go...
3,Air Supply,The One That You Love,"[Now the night has gone., Now the night has go..."
5,Joey Scarbury,"Theme From ""Greatest American Hero"" (Believe I...",NaN
6,Kenny Rogers,I Dont Need You,I don't need you. I don't need friendship. I d...
9,The Oak Ridge Boys,Elvira,"[Elvira, Elvira., My heart's on fire, for Elvi..."
11,The Pointer Sisters,Slow Hand,NaN
12,Kim Carnes,Bette Davis Eyes,"[Her hair is Harlow gold., Her lips sweet surp..."
14,The Manhattan Transfer,Boy From New York City,"[Ooo wah, ooo wah cool, cool kitty., Tell us a..."
16,Marty Balin,Hearts,"[Is everything all right?., I just called to s..."
18,Juice Newton,Queen Of Hearts,"[Midnight, and I'm a-waiting on the twelve-oh-..."


In [49]:
missing_lyrics_df = combined.loc[combined.lyrics.isnull()]
# tmissing_lyrics_df.head(10)

,artist,song,lyrics
5,Joey Scarbury,"Theme From ""Greatest American Hero"" (Believe I...",NaN
11,The Pointer Sisters,Slow Hand,NaN
20,Daryl Hall John Oates,You Make My Dreams,NaN
21,The Moody Blues,Gemini Dream,NaN
22,Ronnie Milsap,(Theres) No Gettin Over Me,NaN
23,Diana Ross & Lionel Richie,Endless Love,NaN
29,Commodores,Lady (You Bring Me Up),NaN
32,Stanley Clarke/George Duke,Sweet Baby,NaN
45,Stars On 45,Medley,NaN
48,Greg Kihn Band,The Breakup Song (They Dont Write Em),NaN


In [51]:
translator = str.maketrans('', '', string.punctuation)

missing_lyrics_df.song = missing_lyrics_df.song.str.translate(translator)
missing_lyrics_df.artist = missing_lyrics_df.artist.str.translate(translator)
missing_lyrics_df.artist = missing_lyrics_df.artist.str.replace('Quavo', 'Quality Control')
missing_lyrics_df = missing_lyrics_df.drop('lyrics', axis =1, errors = 'ignore')

/Users/jennwon/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [13]:
def scrape_genius(artist, song):
    artist_url = artist.replace(' ', '-')
    song_url = song.replace(' ', '-')

    url = f'https://genius.com/{artist_url.lower()}-{song_url.lower()}-lyrics'
    print(url)
    source_code = requests.get(url)
    soup = BeautifulSoup(source_code.text, 'html5lib')
    lyrics = soup.find('div', class_='lyrics').text.strip()

    #cleaning

    lyrics = lyrics.replace('\n', '. ')
    sentences = sent_tokenize(lyrics)
    cleaned = [sentence for sentence in sentences if 
               ('produced' not in sentence.lower()) and
               ('intro' not in sentence.lower()) and
               ('outro' not in sentence.lower()) and
               ('bridge' not in sentence.lower()) and 
               ('verse' not in sentence.lower()) and 
               ('chorus' not in sentence.lower()) and len(sentence) >1]

    #it would be nice to all of this in a dictionary that i can just put into an array and then turn into a pd dataframe

    data = [artist, song, cleaned]
    key =['artist', 'song', 'lyrics']
    song_dict = dict(zip(key, data))
    return song_dict

In [53]:
missing_dict = missing_lyrics_df.to_dict(orient = 'records')
missing_dict[:5]
subset = missing_dict[::-1]
missing_list = []

for track in subset:
    #print(track)
    #print(song)
    artist = track['artist']
    song = track['song']
    try:
        results = scrape_genius(artist, song)
        artist_file = artist.replace(' ', '_')
        song_file = song.replace(' ', '_')
        #print(artist_file, song_file)
        #break
        pd.to_pickle(results, f'third_round/{artist_file}_{song_file}.pkl')
        #print(results)
    except:
        print('sad', track)
        missing_list.append(track)
    #print(results)
    #print(artist, song)
    #break

https://genius.com/zedd-maren-morris-the-middle-lyrics
sad {'artist': 'Zedd Maren Morris', 'song': 'The Middle'}
https://genius.com/zay-hilfigerrr-juju-on-that-beat-tz-anthem-lyrics
https://genius.com/xxxtentacion-infinity-888-lyrics
https://genius.com/xxxtentacion-fk-love-lyrics
sad {'artist': 'XXXTENTACION', 'song': 'Fk Love'}
https://genius.com/tim-mcgraw-the-rest-of-our-life-lyrics
sad {'artist': 'Tim McGraw', 'song': 'The Rest Of Our Life'}
https://genius.com/tim-mcgraw-speak-to-a-girl-lyrics
sad {'artist': 'Tim McGraw', 'song': 'Speak To A Girl'}
https://genius.com/the-weeknd-pray-for-me-lyrics
sad {'artist': 'The Weeknd', 'song': 'Pray For Me'}
https://genius.com/sofia-carson-cameron-boyce-booboo-stewart-chillin-like-a-villain-lyrics
sad {'artist': 'Sofia Carson Cameron Boyce Booboo Stewart', 'song': 'Chillin Like A Villain'}
https://genius.com/schoolboy-q-2-chainz-x-lyrics
sad {'artist': 'ScHoolboy Q 2 Chainz', 'song': 'X'}
https://genius.com/quality-control-takeoff-too-hotty-l

sad {'artist': 'Blessid Union Of Souls ', 'song': 'Light In Your Eyes'}
https://genius.com/-backstreet-boys-everybody-backstreets-back-lyrics
sad {'artist': ' Backstreet Boys', 'song': 'Everybody Backstreets Back'}
https://genius.com/-beenie-man-featuring-mya-girls-dem-sugar-lyrics
sad {'artist': ' Beenie Man Featuring Mya', 'song': 'Girls Dem Sugar'}
https://genius.com/-eminem-cleanin-out-my-closet-lyrics
sad {'artist': ' Eminem', 'song': 'Cleanin Out My Closet'}
https://genius.com/joe-cocker--delta-lady-lyrics
sad {'artist': 'Joe Cocker ', 'song': 'Delta Lady'}
https://genius.com/-isaac-hayes-walk-on-by-lyrics
sad {'artist': ' Isaac Hayes', 'song': 'Walk On By'}
https://genius.com/-sly--the-family-stone-runnin-away-lyrics
sad {'artist': ' Sly  The Family Stone', 'song': 'Runnin Away'}
https://genius.com/-ovis-regular-thang-lyrics
sad {'artist': ' Ovis', 'song': 'Regular Thang'}
https://genius.com/billy-joel--lullabye-goodnight-my-angel-lyrics
sad {'artist': 'Billy Joel ', 'song': 'Lu

sad {'artist': ' Concrete Blonde', 'song': 'Joey'}
https://genius.com/elisa-fiorillo--on-the-way-up-lyrics
sad {'artist': 'Elisa Fiorillo ', 'song': 'On The Way Up'}
https://genius.com/-ralph-tresvant-sensitivity-lyrics
sad {'artist': ' Ralph Tresvant', 'song': 'Sensitivity'}
https://genius.com/-george-michael-freedom-lyrics
sad {'artist': ' George Michael', 'song': 'Freedom'}
https://genius.com/-charlie-rich-my-elusive-dreams-lyrics
sad {'artist': ' Charlie Rich', 'song': 'My Elusive Dreams'}
https://genius.com/-joe-cocker-you-are-so-beautifulits-a-sin-when-you-love-somebody-lyrics
sad {'artist': ' Joe Cocker', 'song': 'You Are So BeautifulIts A Sin When You Love Somebody'}
https://genius.com/-polly-brown-up-in-a-puff-of-smoke-lyrics
sad {'artist': ' Polly Brown', 'song': 'Up In A Puff Of Smoke'}
https://genius.com/america--lonely-people-lyrics
sad {'artist': 'America ', 'song': 'Lonely People'}
https://genius.com/44-when-your-heart-stops-beating-lyrics
https://genius.com/-jibbs-chain

sad {'artist': ' Bo Diddley', 'song': 'You Cant Judge A Book By The Cover'}
https://genius.com/-the-beach-boys-surfin-safari-lyrics
sad {'artist': ' The Beach Boys', 'song': 'Surfin Safari'}
https://genius.com/-brook-benton-lie-to-me-lyrics
sad {'artist': ' Brook Benton', 'song': 'Lie To Me'}
https://genius.com/-sneaker-pimps-spin-spin-sugar-lyrics
sad {'artist': ' Sneaker Pimps', 'song': 'Spin Spin Sugar'}
https://genius.com/the-isley-brothers-featuring-angela-winbush--floatin-on-your-love-lyrics
sad {'artist': 'The Isley Brothers Featuring Angela Winbush ', 'song': 'Floatin On Your Love'}
https://genius.com/merril-bainbridge--mouth-lyrics
sad {'artist': 'Merril Bainbridge ', 'song': 'Mouth'}
https://genius.com/-del-shannon-the-swiss-maid-lyrics
sad {'artist': ' Del Shannon', 'song': 'The Swiss Maid'}
https://genius.com/-booker-t--the-mgs-green-onions-lyrics
sad {'artist': ' Booker T  The MGs', 'song': 'Green Onions'}
https://genius.com/eagles--please-come-home-for-christmas-lyrics
sa

sad {'artist': 'Frankie Valli  The 4 Seasons', 'song': 'Patch Of Blue'}
https://genius.com/-destinys-child-jumpin-jumpin-lyrics
sad {'artist': ' Destinys Child', 'song': 'Jumpin Jumpin'}
https://genius.com/lee-ann-womack--a-little-past-little-rock-lyrics
sad {'artist': 'Lee Ann Womack ', 'song': 'A Little Past Little Rock'}
https://genius.com/tim-mcgraw--where-the-green-grass-grows-lyrics
sad {'artist': 'Tim McGraw ', 'song': 'Where The Green Grass Grows'}
https://genius.com/martika--martikas-kitchen-lyrics
sad {'artist': 'Martika ', 'song': 'Martikas Kitchen'}
https://genius.com/donna-summer--walk-away-lyrics
sad {'artist': 'Donna Summer ', 'song': 'Walk Away'}
https://genius.com/-the-pointer-sisters-could-i-be-dreaming-lyrics
sad {'artist': ' The Pointer Sisters', 'song': 'Could I Be Dreaming'}
https://genius.com/-rick-nelson-and-the-stone-canyon-band-palace-guard-lyrics
sad {'artist': ' Rick Nelson And The Stone Canyon Band', 'song': 'Palace Guard'}
https://genius.com/swv--use-your-

...

sad {'artist': 'Joan Jett  the Blackhearts', 'song': 'Little Liar'}
https://genius.com/lita-ford-duet-with-ozzy-osbourne-close-my-eyes-forever-lyrics
sad {'artist': 'Lita Ford Duet With Ozzy Osbourne', 'song': 'Close My Eyes Forever'}
https://genius.com/-the-beach-boys-kokomo-fromcocktail--lyrics
sad {'artist': ' The Beach Boys', 'song': 'Kokomo FromCocktail '}
https://genius.com/the-pasadenas-tribute-right-on-lyrics
https://genius.com/jimmy-harnen-with-synch-where-are-you-now-lyrics
https://genius.com/huey-lewis--the-news-give-me-the-keys-and-ill-drive-you-crazy-lyrics
sad {'artist': 'Huey Lewis  The News', 'song': 'Give Me The Keys And Ill Drive You Crazy'}
https://genius.com/crosby-stills-nash--young-got-it-made-lyrics
sad {'artist': 'Crosby Stills Nash  Young', 'song': 'Got It Made'}
https://genius.com/johnny-kemp-birthday-suit-from-sing-lyrics
sad {'artist': 'Johnny Kemp', 'song': 'Birthday Suit From Sing'}
https://genius.com/traveling-wilburys-end-of-the-line-lyrics
sad {'artist'

https://genius.com/peter-frampton-baby-i-love-your-way-lyrics
https://genius.com/bachmanturner-overdrive-gimme-your-money-please-lyrics
https://genius.com/tyrone-davis-give-it-up-turn-it-loose-lyrics
https://genius.com/diana-williams-teddy-bears-last-ride-lyrics
sad {'artist': 'Diana Williams', 'song': 'Teddy Bears Last Ride'}
https://genius.com/ronnie-dyson-the-more-you-do-it-the-more-i-like-it-done-to-me-lyrics
sad {'artist': 'Ronnie Dyson', 'song': 'The More You Do It The More I Like It Done To Me'}
https://genius.com/the-salsoul-orchestra-nice-n-naasty-lyrics
sad {'artist': 'The Salsoul Orchestra', 'song': 'Nice N Naasty'}
https://genius.com/natalie-cole-mr-melody-lyrics
https://genius.com/marilyn-mccoo--billy-davis-jr-you-dont-have-to-be-a-star-to-be-in-my-show-lyrics
sad {'artist': 'Marilyn McCoo  Billy Davis Jr', 'song': 'You Dont Have To Be A Star To Be In My Show'}
https://genius.com/john-valenti-anything-you-want-lyrics
sad {'artist': 'John Valenti', 'song': 'Anything You Wan

sad {'artist': 'Gladys Knight And The Pips', 'song': 'I Feel A Song In My HeartDont Burn Down The Bridge'}
https://genius.com/barry-white-youre-the-first-the-last-my-everything-lyrics
https://genius.com/sam-neely-you-can-have-her-lyrics
sad {'artist': 'Sam Neely', 'song': 'You Can Have Her'}
https://genius.com/three-dog-night-play-something-sweet-brickyard-blues-lyrics
https://genius.com/the-pointer-sisters-fairytale-lyrics
sad {'artist': 'The Pointer Sisters', 'song': 'Fairytale'}
https://genius.com/al-wilson-la-la-peace-song-lyrics
sad {'artist': 'Al Wilson', 'song': 'La La Peace Song'}
https://genius.com/rufus-featuring-chaka-khan-you-got-the-love-lyrics
sad {'artist': 'Rufus Featuring Chaka Khan', 'song': 'You Got The Love'}
https://genius.com/al-green-shalala-make-me-happy-lyrics
sad {'artist': 'Al Green', 'song': 'ShaLaLa Make Me Happy'}
https://genius.com/the-hues-corporation-rockin-soul-lyrics
sad {'artist': 'The Hues Corporation', 'song': 'Rockin Soul'}
https://genius.com/the-

sad {'artist': 'Baby Boy Da Prince Featuring Lil Boosie', 'song': 'The Way I Live'}
https://genius.com/bow-wow-featuring-tpain--johnta-austin-outta-my-system-lyrics
sad {'artist': 'Bow Wow Featuring TPain  Johnta Austin', 'song': 'Outta My System'}
https://genius.com/gym-class-heroes-featuring-patrick-stump--cupids-chokeholdbreakfast-in-america-lyrics
sad {'artist': 'Gym Class Heroes Featuring Patrick Stump ', 'song': 'Cupids ChokeholdBreakfast In America'}
https://genius.com/diddy-featuring-keyshia-cole-last-night-lyrics
sad {'artist': 'Diddy Featuring Keyshia Cole', 'song': 'Last Night'}
https://genius.com/gwen-stefani-featuring-akon-the-sweet-escape-lyrics
sad {'artist': 'Gwen Stefani Featuring Akon', 'song': 'The Sweet Escape'}
https://genius.com/r-kelly-or-bow-wow-featuring-ti--tpain-im-a-flirt-lyrics
sad {'artist': 'R Kelly Or Bow Wow Featuring TI  TPain', 'song': 'Im A Flirt'}
https://genius.com/pnk-u--ur-hand-lyrics
sad {'artist': 'Pnk', 'song': 'U  Ur Hand'}
https://genius.com

sad {'artist': ' Diana Ross  Lionel Richie', 'song': 'Endless Love'}
https://genius.com/ronnie-milsap-theres-no-gettin-over-me-lyrics
https://genius.com/the-moody-blues-gemini-dream-lyrics
sad {'artist': 'The Moody Blues', 'song': 'Gemini Dream'}
https://genius.com/daryl-hall-john-oates-you-make-my-dreams-lyrics
sad {'artist': 'Daryl Hall John Oates', 'song': 'You Make My Dreams'}
https://genius.com/the-pointer-sisters-slow-hand-lyrics
sad {'artist': 'The Pointer Sisters', 'song': 'Slow Hand'}
https://genius.com/joey-scarbury-theme-from-greatest-american-hero-believe-it-or-not-lyrics
sad {'artist': 'Joey Scarbury', 'song': 'Theme From Greatest American Hero Believe It or Not'}


In [57]:
song_df = pd.DataFrame(pd.read_pickle('scraped_lyrics_816.pkl'))
song_df.head()

,artist,lyrics,song
0,The Cascades,The last leaf clings to the bough. Just one le...,The Last Leaf
0,Keith Urban,I woke up early this morning around 4am. With ...,Youll Think Of Me
0,BJ Thomas,Who's the one who tied you shoe when you were ...,Mama
0,The Mixtures,Riding along my pushbike honey. When I noticed...,Pushbike Song
0,Greg Kihn Band,Oh Oh Oh Oh. Where were you when I needed you?...,Jeopardy


In [54]:
path_names = glob('third_round/*.pkl')
for path in path_names:
    df = pd.DataFrame(pd.read_pickle(str(path)))
    df.head()
    break

In [58]:
path_names = glob('third_round/*.pkl')

for path in path_names:
    try:
        
        song = pd.DataFrame(pd.read_pickle(str(path)))
        combined = song.groupby(['artist', 'song'])['lyrics'].apply(' '.join).reset_index()
        song_df = song_df.append(combined)
    except:
        missing.append(path)
        print(missing)
        #break
    
song_df

,artist,lyrics,song
0,The Cascades,The last leaf clings to the bough. Just one le...,The Last Leaf
0,Keith Urban,I woke up early this morning around 4am. With ...,Youll Think Of Me
0,BJ Thomas,Who's the one who tied you shoe when you were ...,Mama
0,The Mixtures,Riding along my pushbike honey. When I noticed...,Pushbike Song
0,Greg Kihn Band,Oh Oh Oh Oh. Where were you when I needed you?...,Jeopardy
0,Expose,More than ever you're on my mind. Being lonely...,I Wish The Phone Would Ring
0,Carpenters,The hardest thing I've ever done. Is keep beli...,I Need To Be In Love
0,Soul II Soul,"I can see, i can see, i can see, i can see. I ...",A Dreams A Dream
0,Vitamin Z,Walking out tonight in the street lights I can...,Burning Flame
0,Herbie Hancock,[Instrumental],Chameleon


In [60]:
pd.to_pickle(song_df, 'third_round_results_lyrics.pkl')

song_df = pd.DataFrame(pd.read_pickle('third_round_results_lyrics.pkl'))
len(song_df)

18007